In [48]:
import h5py, os
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
from skimage import exposure
from datetime import datetime, timedelta
import glob
import requests
import time

## Checking tiles before dowloading images of VNP46A1

In [49]:
points_ = pd.read_csv('..\\rawpoints_coordinates\\coordinates_processed_no_duplication.csv')
points_['Date_Mscan'] = pd.to_datetime(points_['Date_Mscan'], format='%Y-%m-%d')
points_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290117 entries, 0 to 290116
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  290117 non-null  int64         
 1   Lat_GMTCO   290117 non-null  float64       
 2   Lon_GMTCO   290117 non-null  float64       
 3   Temp_BB     290117 non-null  int64         
 4   Date_Mscan  290117 non-null  datetime64[ns]
 5   tile        290117 non-null  object        
 6   duplicated  290117 non-null  bool          
 7   id          290117 non-null  int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 15.8+ MB


In [50]:
os.chdir("..\\linksimages_nasa")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv['link'] = combined_csv['Details'].str.split(';', 0).str[1]
combined_csv['name'] = combined_csv['Details'].str.split(';', 0).str[2]
combined_csv['day'] = combined_csv['name'].str.split('.', 0).str[1]
combined_csv['tile'] = combined_csv['name'].str.split('.', 0).str[2]
combined_csv = combined_csv[['link', 'name','tile', 'day']]
combined_csv

C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\454996658.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  combined_csv['link'] = combined_csv['Details'].str.split(';', 0).str[1]
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\454996658.py:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  combined_csv['name'] = combined_csv['Details'].str.split(';', 0).str[2]
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\454996658.py:10: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  combined_csv['day'] = combined_csv['name'].str.split('.', 0).str[1]
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\454996658.py:11: FutureWarning: In a future version of pandas all arguments of StringMethods.split 

link  \
0    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
1    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
2    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
3    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
4    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
..                                                 ...   
607  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
608  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
609  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
610  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
611  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   

                                             name    tile       day  
0    VNP46A1.A2019213.h33v03.001.2019214124816.h5  h33v03  A2019213  
1    VNP46A1.A2019213.h07v08.001.2019214114917.h5  h07v08  A2019213  
2    VNP46A1.A2019213.h16v10.001.2019214120611.h5  h16v10  A2019213  
3    VNP46A1.A2019213.h16v07.001.2019214114451.h5  h16v07  A2019213  
4    VNP46A1.A2019213.h05v06.001.2019214115243.h5  h05v06  A2019213  
..                                            ...     ...       ...  
607  VNP46A1.A2019243.h19v14.001.2019244125611.h5  h19v14  A2019243  
608  VNP46A1.A2019243.h29v17.001.2019244133246.h5  h29v17  A2019243  
609  VNP46A1.A2019243.h08v06.001.2019244121148.h5  h08v06  A2019243  
610  VNP46A1.A2019243.h35v14.001.2019244132924.h5  h35v14  A2019243  
611  VNP46A1.A2019243.h29v10.001.2019244123357.h5  h29v10  A2019243  

[18292 rows x 4 columns]

In [51]:
# Iteration to get files by tile having  at least one flare.

init_date = datetime.strptime('2019-08-01', '%Y-%m-%d')
end_date = datetime.strptime('2019-08-31', '%Y-%m-%d')

day_year = 213

df_images= pd.DataFrame()

while init_date.date() <= end_date.date():
    #Identified points and its tiles to image dowload
    tiles = points_[points_['Date_Mscan'] == init_date]['tile'].unique()   
    groupday = combined_csv[combined_csv["day"]=="A2019"+str(day_year)]
    
    # Getting just images with identified flares
    df_download = groupday[groupday['tile'].isin(tiles)]
    df_images = df_images.append(df_download)
    
    init_date = init_date + timedelta(days=1)
    day_year+=1
    
df_images

C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\3300178811.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_images = df_images.append(df_download)
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\3300178811.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_images = df_images.append(df_download)
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\3300178811.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_images = df_images.append(df_download)
C:\Users\osmary.camila\AppData\Local\Temp\ipykernel_51276\3300178811.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_images = df_images.append(d

link  \
2    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
3    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
7    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
8    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
9    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
..                                                 ...   
594  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
600  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
601  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
609  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
611  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   

                                             name    tile       day  
2    VNP46A1.A2019213.h16v10.001.2019214120611.h5  h16v10  A2019213  
3    VNP46A1.A2019213.h16v07.001.2019214114451.h5  h16v07  A2019213  
7    VNP46A1.A2019213.h10v11.001.2019214115646.h5  h10v11  A2019213  
8    VNP46A1.A2019213.h24v02.001.2019214130358.h5  h24v02  A2019213  
9    VNP46A1.A2019213.h30v08.001.2019214124414.h5  h30v08  A2019213  
..                                            ...     ...       ...  
594  VNP46A1.A2019243.h29v03.001.2019244130210.h5  h29v03  A2019243  
600  VNP46A1.A2019243.h22v11.001.2019244130828.h5  h22v11  A2019243  
601  VNP46A1.A2019243.h32v05.001.2019244125530.h5  h32v05  A2019243  
609  VNP46A1.A2019243.h08v06.001.2019244121148.h5  h08v06  A2019243  
611  VNP46A1.A2019243.h29v10.001.2019244123357.h5  h29v10  A2019243  

[5862 rows x 4 columns]

In [52]:
df_images.to_csv("..\\linksimages_nasa\\complete_links\\links_download_processed.csv", index=False)

## Starting downloading pre steps

In [54]:
df_download = pd.read_csv('..\\linksimages_nasa\\complete_links\\links_download_processed.csv')
df_download.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5862 entries, 0 to 5861
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    5862 non-null   object
 1   name    5862 non-null   object
 2   tile    5862 non-null   object
 3   day     5862 non-null   object
dtypes: object(4)
memory usage: 183.3+ KB


In [55]:
df_download

link  \
0     https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
1     https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
2     https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
3     https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
4     https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
...                                                 ...   
5857  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5858  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5859  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5860  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5861  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   

                                              name    tile       day  
0     VNP46A1.A2019213.h16v10.001.2019214120611.h5  h16v10  A2019213  
1     VNP46A1.A2019213.h16v07.001.2019214114451.h5  h16v07  A2019213  
2     VNP46A1.A2019213.h10v11.001.2019214115646.h5  h10v11  A2019213  
3     VNP46A1.A2019213.h24v02.001.2019214130358.h5  h24v02  A2019213  
4     VNP46A1.A2019213.h30v08.001.2019214124414.h5  h30v08  A2019213  
...                                            ...     ...       ...  
5857  VNP46A1.A2019243.h29v03.001.2019244130210.h5  h29v03  A2019243  
5858  VNP46A1.A2019243.h22v11.001.2019244130828.h5  h22v11  A2019243  
5859  VNP46A1.A2019243.h32v05.001.2019244125530.h5  h32v05  A2019243  
5860  VNP46A1.A2019243.h08v06.001.2019244121148.h5  h08v06  A2019243  
5861  VNP46A1.A2019243.h29v10.001.2019244123357.h5  h29v10  A2019243  

[5862 rows x 4 columns]

In [56]:
# list downloaded files already
res = os.listdir("..\\h5_files")
df_download_new = df_download[~df_download['name'].isin(res)]
df_download_new

link  \
40    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
41    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
42    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
43    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
44    https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
...                                                 ...   
5857  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5858  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5859  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5860  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   
5861  https://ladsweb.modaps.eosdis.nasa.gov/api/v2/...   

                                              name    tile       day  
40    VNP46A1.A2019213.h25v03.001.2019214120949.h5  h25v03  A2019213  
41    VNP46A1.A2019213.h34v16.001.2019214125412.h5  h34v16  A2019213  
42    VNP46A1.A2019213.h20v05.001.2019214120249.h5  h20v05  A2019213  
43    VNP46A1.A2019213.h31v09.001.2019214125409.h5  h31v09  A2019213  
44    VNP46A1.A2019213.h11v14.001.2019214123126.h5  h11v14  A2019213  
...                                            ...     ...       ...  
5857  VNP46A1.A2019243.h29v03.001.2019244130210.h5  h29v03  A2019243  
5858  VNP46A1.A2019243.h22v11.001.2019244130828.h5  h22v11  A2019243  
5859  VNP46A1.A2019243.h32v05.001.2019244125530.h5  h32v05  A2019243  
5860  VNP46A1.A2019243.h08v06.001.2019244121148.h5  h08v06  A2019243  
5861  VNP46A1.A2019243.h29v10.001.2019244123357.h5  h29v10  A2019243  

[5822 rows x 4 columns]

## Dowloading image

In [ ]:
def download_file(link, name):
    auth_token='b3NtYXJ5MTg0OmIzTnRZWEo1UUdGc2RXNXZjeTUxZEdad2NpNWxaSFV1WW5JPToxNjU4NzU3NTc1OjY4N2Q5MGFiOWVmNmRlNjM4OTMyNWY0MzcwY2NlNDFlMWNhNDc2Mzg'
    hed = {'Authorization': 'Bearer ' + auth_token, 'X-Requested-With' : 'XMLHttpRequest'}
     
    response = requests.get(link, headers=hed)
    with open("..\\h5_files\\" + name, 'wb') as f:
        time.sleep(5)
        f.write(response.content)        
        f.close()
        
        return True
    return False
    
df_download_new['processed'] = df_download_new.apply(lambda x: download_file(x['link'], x['name']), axis=1)

## Redownloading files with errors

In [ ]:
df_download_new[~df_download_new['processed']]

In [21]:
def download_file(link, name):
    auth_token='b3NtYXJ5MTg0OmIzTnRZWEo1UUdGc2RXNXZjeTUxZEdad2NpNWxaSFV1WW5JPToxNjU4NzU3NTc1OjY4N2Q5MGFiOWVmNmRlNjM4OTMyNWY0MzcwY2NlNDFlMWNhNDc2Mzg'
    hed = {'Authorization': 'Bearer ' + auth_token, 'X-Requested-With' : 'XMLHttpRequest'}
     
    response = requests.get(link, headers=hed)
    with open("C:\\Users\\Mary Camila\\Downloads\\Dataset - VNP46A1 and VNP46A2\\VNP46A1\\H5 Files\\temp\\A2019214\\" + name, 'wb') as f:
        time.sleep(5)
        f.write(response.content)        
        f.close()
        
        return True
    return False
    
files_errors['processed'] = files_errors.apply(lambda x: download_file(x['link'], x['name']), axis=1)